<a href="https://colab.research.google.com/github/seobando/TradingBot/blob/main/2_kpis_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/1 Formal Training/2 Msc. Ciencia de los datos/Semestre IV/Trader

/content/drive/MyDrive/1 Formal Training/2 Msc. Ciencia de los datos/Semestre IV/Trader


In [ ]:
#!pip install pandas_ta

In [ ]:
import random
import torch
import pandas as pd
import numpy as np

from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def kpis(df,short_period,long_period,file_name):
  # Simple Moving Average
  df['MA_short'] = df['Close'].rolling(short_period).mean()
  df['MA_long'] = df['Close'].rolling(long_period).mean()
  # Exponential Moving Average
  df['EMA_short'] = df['Close'].ewm(span=short_period, adjust=False).mean()
  df['EMA_long'] = df['Close'].ewm(span=long_period, adjust=False).mean()
  # Moving average convergence / divergence
  #https://tcoil.info/compute-macd-indicator-for-stocks-with-python/
  df['EMA_short_MACD'] = df['Close'].ewm(span=12, adjust=False).mean()
  df['EMA_long_MACD'] = df['Close'].ewm(span=26, adjust=False).mean()
  df['MACD'] = df['EMA_short_MACD'] - df['EMA_long_MACD']
  df['MACD_signal'] = df['MACD'].ewm(span=9, min_periods=9).mean()
  df['MACD_hist'] = df['MACD'] - df['MACD_signal']
  # Bias
  df['BIAS_short'] = (df['Close'] - df['MA_short'])/ df['MA_short'] * 100
  df['BIAS_long'] = (df['Close'] - df['MA_long'])/ df['MA_long'] * 100
  # Volatility Volume Ratio
  df["Previous Close"] = df["Close"].shift(-1)
  df["u"] = np.where((df["Previous Close"] - df["Close"]) > 0, df["Volume"],0)
  df["d"] = np.where((df["Previous Close"] - df["Close"]) < 0, df["Volume"],0)
  df["v"] = np.where((df["Previous Close"] - df["Close"]) == 0, df["Volume"],0)
  df["VR"] = (df["u"].rolling(short_period).sum() + 1/2*df["v"].rolling(short_period).sum())/df["d"].rolling(short_period).sum() + 1/2*df["v"].rolling(short_period).sum() * 100
  # On Balance Volume
  #https://www.investopedia.com/terms/o/onbalancevolume.asp
  ## Rules
  size = len(df["Close"])
  OBV = []
  OBV.append(0)
  for i in range(1, size):
      #1. If today's closing price is higher than yesterday's closing price, then: Current OBV = Previous OBV + today's volume
      if df["Close"][i] > df["Close"][i-1]: #If the closing price is above the prior close price 
            OBV.append(OBV[-1] + df["Volume"][i]) #then: Current OBV = Previous OBV + Current Volume
      #2. If today's closing price is lower than yesterday's closing price, then: Current OBV = Previous OBV - today's volume      
      elif df["Close"][i] < df["Close"][i-1]:
            OBV.append( OBV[-1] - df["Volume"][i])
      #3. If today's closing price equals yesterday's closing price, then: Current OBV = Previous OBV      
      else:
            OBV.append(OBV[-1])

  df['OBV'] = OBV

  # Delete NA Values
  #df = df.dropna()
  # Pick required fields and save
  required_fields = ['Date', 'Open', 'High','Low', 'Close', 'Volume', 'MA_short', 'MA_long', 'EMA_short','EMA_long','EMA_short_MACD','EMA_long_MACD', 'MACD','MACD_signal','MACD_hist', 'BIAS_short', 'BIAS_long', 'VR', 'OBV']
  df[required_fields].to_csv(file_name)

In [ ]:
# Get daily data with KPIS 
long_period = 200
short_period = 50
frequency = "daily"
#df = pd.read_csv("data/BTC-USD.csv")
#df = pd.read_csv("data/AAPL.csv")
#file_name = "data/data_daily_AAPL.csv"

# Expected data
datasets = [
            "data/data_daily_BTC-USD.csv",
            "data/data_daily_AAPL.csv",
            "data/data_daily_SPY.csv",
            "data/data_daily_^XAU.csv"
            ]

for d in datasets:
  file_name = d
  f = "data/" + d.split("_")[2]
  df = pd.read_csv(f)            
  kpis(df,short_period,long_period,file_name)

In [ ]:
# Check data
for i,value in enumerate(datasets):
  print("------------------------", value, "------------------------")
  pd.read_csv(datasets[i]).info()

------------------------ data/data_daily_BTC-USD.csv ------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1518 entries, 0 to 1517
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      1518 non-null   int64  
 1   Date            1518 non-null   object 
 2   Open            1518 non-null   float64
 3   High            1518 non-null   float64
 4   Low             1518 non-null   float64
 5   Close           1518 non-null   float64
 6   Volume          1518 non-null   int64  
 7   MA_short        1469 non-null   float64
 8   MA_long         1319 non-null   float64
 9   EMA_short       1518 non-null   float64
 10  EMA_long        1518 non-null   float64
 11  EMA_short_MACD  1518 non-null   float64
 12  EMA_long_MACD   1518 non-null   float64
 13  MACD            1518 non-null   float64
 14  MACD_signal     1510 non-null   float64
 15  MACD_hist       1510 non-null   float64
 16  